In [5]:
import pandas as pd

In [8]:
data = pd.read_csv("mergedtransport_output.csv")
data 

,transport,route_name,route_link,busname,bustype,departing_time,duration,reaching_time,star_rating,price,seats_available
0,ASTC,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 147480,Bharat Benz A/C Seater (2+2),05:00,04h 00m,09:00,3.6,270.0,NaN
1,ASTC,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 158332,Bharat Benz A/C Seater (2+2),07:10,03h 50m,11:00,4.9,298.0,NaN
2,ASTC,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 153223,Bharat Benz A/C Seater (2+2),09:30,05h 00m,14:30,3.9,298.0,NaN
3,ASTC,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 157959,Bharat Benz A/C Seater (2+2),14:00,04h 30m,18:30,4.5,298.0,NaN
4,ASTC,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 154930,Volvo AC Seater 2+2,15:45,04h 15m,20:00,4.2,298.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7157,WBTC,Kolkata to Asansol (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...,Tulsi Travel,NON A/C Seater / Sleeper (2+2),14:45,02h 05m,16:50,3.2,899.0,NaN
7158,WBTC,Barasat (West Bengal) to Debra,https://www.redbus.in/bus-tickets/barasat-west...,SBSTC-BARASAT - MIDNAPUR - 08:30 (MID DEPOT) -...,Non AC Seater (2+3),08:30,15h 30m,00:00,4.1,103.0,30 Seats available
7159,WBTC,Barasat (West Bengal) to Nimtouri,https://www.redbus.in/bus-tickets/barasat-west...,SBSTC-BARASAT - DIGHA - VIA - NIBEDITA SETU - ...,Non AC Seater (2+3),11:15,12h 45m,00:00,3.6,93.0,48 Seats available
7160,WBTC,Habra to Kolaghat,https://www.redbus.in/bus-tickets/habra-to-kol...,WBTC (CTC) HABRA-DIGHA via Bally - 26|12:45,Non AC Seater (2+3),12:45,02h 55m,15:40,3.8,87.0,NaN


In [10]:
import mysql.connector

# Connect to MySQL server
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)

# Create a cursor object
mycursor = mydb.cursor()

# Drop the existing database (if required)
mycursor.execute("DROP DATABASE IF EXISTS redbus")

# Create a new database
mycursor.execute("CREATE DATABASE redbus")

# Select the newly created database
mycursor.execute("USE redbus")

# Create the redbus table with TEXT fields
mycursor.execute('''
CREATE TABLE IF NOT EXISTS redbus_data (
    id INT PRIMARY KEY AUTO_INCREMENT,
    transport TEXT,
    route_name TEXT,
    route_link TEXT,
    busname TEXT,
    bustype TEXT,
    departing_time TEXT,
    duration TEXT,
    reaching_time TEXT,
    star_rating FLOAT,
    price DECIMAL(10, 2),
    seats_available INT
)
''')


mydb.commit()

# Close the cursor and connection
mycursor.close()
mydb.close()


In [11]:
import pandas as pd
import mysql.connector

# Load the data from CSV
data = pd.read_csv("mergedtransport_output.csv")

# Handle missing values
data.fillna({
    'transport': '',
    'route_name': '',  # Assuming missing route names can be empty strings
    'route_link': '',  # Assuming missing route links can be empty strings
    'busname': '',  # Assuming missing bus names can be empty strings
    'bustype': '',  # Assuming missing bus types can be empty strings
    'departing_time': '',  # Assuming missing departing times can be set to '00:00:00'
    'duration': '0',  # Assuming missing durations can be set to '0'
    'reaching_time': '',  # Assuming missing reaching times can be set to '00:00:00'
    'star_rating': 0.0,  # Assuming missing star ratings can be set to 0.0
    'price': 0.0,  # Assuming missing prices can be set to 0.0
    'seats_available': 0  # Assuming missing seat availability can be set to 0
}, inplace=True)

# Convert 'departing_time' and 'reaching_time' to proper format
#data['departing_time'] = pd.to_datetime(data['departing_time'], errors='coerce')#.dt.time.fillna(pd.Timestamp('00:00:00').time())
#data['reaching_time'] = pd.to_datetime(data['reaching_time'], errors='coerce')#.dt.time.fillna(pd.Timestamp('00:00:00').time())

# Connect to MySQL server
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="redbus"
)

# Create a cursor object
mycursor = mydb.cursor()

# Iterate over the DataFrame and insert values into the table
for index, row in data.iterrows():
    sql = '''
    INSERT INTO redbus_data (transport, route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''
    values = (row['transport'],row['route_name'], row['route_link'], row['busname'], row['bustype'], row['departing_time'], row['duration'], row['reaching_time'], row['star_rating'], row['price'], row['seats_available'])
    
    mycursor.execute(sql, values)

# Commit the transaction
mydb.commit()

# Close the cursor and connection
mycursor.close()
mydb.close()
